# Serão realizados vários procedimentos de estimação de dados faltantes levando em conta a natureza do dado faltante e em seguida será realizado uma previsão de um tipo de alga marinha pelo algoritmo de Árvore de Regressão.

In [2]:
require(rpart)
require(rpart.plot)
require(DMwR)
require("TeachingSampling")
require(ggplot2)
require(nortest)
require(e1071)
require(mice)

Loading required package: rpart.plot

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'rpart.plot'"
Loading required package: TeachingSampling

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'TeachingSampling'"


In [ ]:
data(algae)

In [ ]:
#Observações com muitas variáveis faltantes, como são apenas 2 observações retirei dos dados.
algae[c(62,199),] 
dados=algae[-c(62,199),]

In [ ]:
head(dados)

In [ ]:
#Proporção de dados faltantes
nrow(dados[!complete.cases(dados),])/nrow(dados)

# Função que põem os nomes das linhas certas

In [ ]:
AjeitaDados=function(y){
y=algae[-c(62,199),]
x=apply(y,1,function(x) sum(is.na(x)));x
x=data.frame(x)
acocha=as.numeric(rownames(x));acocha
length(acocha)
acocha[62:197]=acocha[62:197]-1
acocha[198]=acocha[198]-2
acocha
rownames(x)=as.character(acocha);rownames(x)
x=cbind(rownames(x),x);x
rownames(y)=as.character(x[,1])
return(y)
}

In [ ]:
dados=AjeitaDados(dados)

In [ ]:
##Valores faltantes nas linhas
x=apply(dados,1,function(x) sum(is.na(x)))
x[x!=0]

In [ ]:
##Valores faltantes nas colunas
apply(dados,2,function(x) sum(is.na(x)))

# Pré-Processamento de Dados e Avaliação de modelos por Àrvore de Regressão com alguns métodos de estimação de dados faltantes.

In [ ]:
#Estimando os valores faltantes pelo método do pacote rpart
metricas=c()
set.seed(7)
for(i in 1:10){
amostra=S.SI(nrow(dados),floor(nrow(dados)*0.7))
treino=dados[amostra,]
teste=dados[-amostra,]
modelo1=rpart(a1 ~ ., data = treino[, 1:12])
rpart.plot(modelo1, type=0, varlen=10)
##Avaliação da performance do modelo
metricas=append(metricas,mean((teste$a1-predict(modelo1,teste[,1:11]))^2))
}

In [ ]:
metricas
m1=cbind(mean(metricas),sd(metricas))

In [ ]:
##Estimação dos valores faltantes tirando as linhas com valores faltantes
dados=AjeitaDados(dados)
dados2=dados[complete.cases(dados),]
metricas=c()
set.seed(7)
for(i in 1:10){
amostra1=S.SI(nrow(dados2),floor(nrow(dados2)*0.7))
treino=dados2[amostra1,]
teste=dados2[-amostra1,]
modelo2 <- rpart(a1 ~ ., data = treino[, 1:12])
rpart.plot(modelo2, type=0, varlen=10)
##Avaliação do modelo
metricas=append(metricas,mean((teste$a1-predict(modelo2,teste[1:11]))^2))
}

In [ ]:
metricas
m2=cbind(mean(metricas),sd(metricas))
m=rbind(m1,m2)

# Estimação dos valores faltantes pela média, mediana.

In [ ]:
dados=AjeitaDados(dados)
##Variáveis que tem valores faltantes
apply(dados,2,function(x) sum(is.na(x)))

# Variável mxPH

In [ ]:
(ggplot(dados)+aes(mxPH)+geom_histogram(aes(y=..density..),color="black",fill="blue")
+geom_rug()+geom_density())

In [ ]:
skewness(dados$mxPH,na.rm=T)
kurtosis(dados$mxPH,na.rm=T)

In [ ]:
(ggplot(dados)+aes(1,mxPH)+geom_boxplot(fill="blue",color="black")
  +geom_rug()+geom_hline(aes(yintercept=mean(dados$mxPH,na.rm=T),color="red")))

In [ ]:
##Proporção dos outliers
6/198

In [ ]:
qqnorm(dados$mxPH)
qqline(dados$mxPH,col="red")

In [ ]:
shapiro.test(dados$mxPH)
lillie.test(dados$mxPH)

# Conclusão: a variável mxPH pode ser considerada com distribuição normal com evidências do histograma e sua densidade estimada, assimetria e curtose, box-plot e a proporção de outlier chega perto de 5%, e um dos dois testes de hipóteses realizados não-rejeitam normalidade a nível de 1%, com isso o valor a ser imputado será a média.

In [ ]:
x=which(is.na(dados$mxPH))
dados$mxPH[x]=mean(dados$mxPH,na.rm=T)

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

# Variável mnO2

In [ ]:
(ggplot(dados)+aes(mnO2)+geom_histogram(aes(y=..density..),color="black",fill="blue")
  +geom_rug()+geom_density())

In [ ]:
skewness(dados$mnO2,na.rm=T)
kurtosis(dados$mnO2,na.rm=T)

In [ ]:
(ggplot(dados)+aes(1,mnO2)+geom_boxplot(fill="blue",color="black")
  +geom_rug()+geom_hline(aes(yintercept=mean(dados$mnO2,na.rm=T),color="red")))

In [ ]:
qqnorm(dados$mnO2)
qqline(dados$mnO2,col="red")

In [ ]:
shapiro.test(dados$mnO2)
lillie.test(dados$mnO2)

# Conclusão: a variável mnO2 não pode ser considerada com distribuição normal pelas evidências do histograma e sua densidade estimada, assimetria e curtose, box-plot e os dois testes de hipóteses realizados rejeitaram normalidade, com isso o valor a ser imputado será a mediana.

In [ ]:
x=which(is.na(dados$mnO2))
dados$mnO2[x]=median(dados$mnO2,na.rm=T)

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

# Variável Cl

In [ ]:
(ggplot(dados)+aes(Cl)+geom_histogram(aes(y=..density..),color="black",fill="blue")
  +geom_rug()+geom_density())

In [ ]:
skewness(dados$Cl,na.rm=T)
kurtosis(dados$Cl,na.rm=T)

In [ ]:
(ggplot(dados)+aes(1,Cl)+geom_boxplot(fill="blue",color="black")
  +geom_rug()+geom_hline(aes(yintercept=mean(dados$Cl,na.rm=T),color="red")))

In [ ]:
qqnorm(dados$Cl)
qqline(dados$Cl,col="red")

In [ ]:
shapiro.test(dados$Cl)
lillie.test(dados$Cl)

# Conclusão: a variável Cl não pode ser considerada com distribuição normal pelas evidências do histograma e sua densidade estimada, assimetria e calda bem mais pesado que a normal, box-plot e os dois testes de hipóteses realizados rejeitaram normalidade, com isso o valor a ser imputado será a mediana por ser mais robusta nessa situação.

In [ ]:
x=which(is.na(dados$Cl));x
dados$Cl[x]=median(dados$Cl,na.rm=T)

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

# Variável PO4

In [ ]:
(ggplot(dados)+aes(PO4)+geom_histogram(aes(y=..density..),color="black",fill="blue")
  +geom_rug()+geom_density())

In [ ]:
skewness(dados$PO4,na.rm=T)
kurtosis(dados$PO4,na.rm=T)

In [ ]:
(ggplot(dados)+aes(1,PO4)+geom_boxplot(fill="blue",color="black")
  +geom_rug()+geom_hline(aes(yintercept=mean(dados$PO4,na.rm=T),color="red")))

In [ ]:
qqnorm(dados$PO4)
qqline(dados$PO4,col="red")

In [ ]:
shapiro.test(dados$PO4)
lillie.test(dados$PO4)

In [ ]:
x=which(is.na(dados$PO4))
dados$PO4[x]=median(dados$PO4,na.rm=T)

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

# Conclusão: a variável PO4 não pode ser considerada com distribuição normal pelas evidências do histograma e sua densidade estimada, assimetria e calda bem mais pesado que a normal, box-plot e os dois testes de hipóteses realizados rejeitaram normalidade, com isso o valor a ser imputado será a mediana por ser mais robusta nessa situação.¶

# Variável Chla

In [ ]:
(ggplot(dados)+aes(Chla)+geom_histogram(aes(y=..density..),color="black",fill="blue")
  +geom_rug()+geom_density())

In [ ]:
skewness(dados$Chla,na.rm=T)
kurtosis(dados$Chla,na.rm=T)

In [ ]:
(ggplot(dados)+aes(1,Chla)+geom_boxplot(fill="blue",color="black")
  +geom_rug()+geom_hline(aes(yintercept=mean(dados$Chla,na.rm=T),color="red")))

In [ ]:
qqnorm(dados$Chla)
qqline(dados$Chla,col="red")

In [ ]:
shapiro.test(dados$Chla)
lillie.test(dados$Chla)

# Conclusão: a variável Chla não pode ser considerada com distribuição normal pelas evidências do histograma e sua densidade estimada, assimetria e calda bem mais pesado que a normal, box-plot e os dois testes de hipóteses realizados rejeitaram normalidade, com isso o valor a ser imputado será a mediana por ser mais robusta nessa situação.

In [ ]:
x=which(is.na(dados$Chla));x
dados$Chla[x]=median(dados$Chla,na.rm=T)

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

In [ ]:
metricas=c()
set.seed(7)
for(i in 1:10){
amostra=S.SI(nrow(dados),floor(nrow(dados)*0.7))
treino=dados[amostra,]
teste=dados[-amostra,]
modelo3=rpart(a1 ~ ., data = treino[, 1:12])
rpart.plot(modelo3, type=0, varlen=10)
##Avaliação da performance do modelo
metricas=append(metricas,mean((teste$a1-predict(modelo3,teste[,1:11]))^2))
}

In [ ]:
metricas
m3=cbind(mean(metricas),sd(metricas))
m=rbind(m,m3)

# Estimando pela correlação entre as variáveis

In [ ]:
dados=AjeitaDados(dados)
apply(dados,2,function(x) sum(is.na(x)))

# Matriz de correlações

In [ ]:
symnum(cor(dados[,4:18],use="complete.obs"))

# Variável PO4

In [ ]:
#0.91196460
ggplot(dados)+aes(oPO4,PO4)+geom_point()+geom_smooth(method="lm",se=F)

In [ ]:
x=lm(dados$PO4 ~ dados$oPO4, data=dados)
summary(x)

In [ ]:
a=which(is.na(dados$PO4))
dados$PO4[a]=x$coefficients%*%t(t(c(1,dados$oPO4[a])))

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

# Variável Chla

In [ ]:
#0.443046449
ggplot(dados)+aes(mxPH,Chla)+geom_point()+geom_smooth(method="lm",se=F)

In [ ]:
x=lm(dados$Chla ~ dados$mxPH, data=dados)
summary(x)

In [ ]:
a=which(is.na(dados$Chla));a
dados$Chla[a]=cbind(rep(1,length(a)),dados$mxPH[a])%*%t(t(x$coefficients))

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

# Variável mxPH

In [ ]:
#0.443046449
ggplot(dados)+aes(Chla,mxPH)+geom_point()+geom_smooth(method="lm",se=F)

In [ ]:
x=lm(dados$mxPH ~ dados$Chla, data=dados)
summary(x)

In [ ]:
a=which(is.na(dados$mxPH))
dados$mxPH[a]=cbind(rep(1,length(a)),dados$Chla[a])%*%t(t(x$coefficients))

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

# Variável Cl

In [ ]:
#0.4590139
ggplot(dados)+aes(PO4,Cl)+geom_point()+geom_smooth(method="lm",se=F)

In [ ]:
x=lm(dados$Cl ~ dados$PO4, data=dados)
summary(x)

In [ ]:
a=which(is.na(dados$Cl))
dados$Cl[a]=cbind(rep(1,length(a)),dados$PO4[a])%*%t(t(x$coefficients))

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

# Variável mnO2

In [ ]:
#-0.4889931
ggplot(dados)+aes(PO4,mnO2)+geom_point()+geom_smooth(method="lm",se=F)

In [ ]:
x=lm(dados$mnO2 ~ dados$PO4, data=dados)
summary(x)

In [ ]:
a=which(is.na(dados$mnO2))
dados$mnO2[a]=cbind(rep(1,length(a)),dados$PO4[a])%*%t(t(x$coefficients))

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

In [ ]:
metricas=c()
set.seed(7)
for(i in 1:10){
amostra=S.SI(nrow(dados),floor(nrow(dados)*0.7))
treino=dados[amostra,]
teste=dados[-amostra,]
modelo4=rpart(a1 ~ ., data = treino[, 1:12])
rpart.plot(modelo4, type=0, varlen=10)
##Avaliação da performance do modelo
metricas=append(metricas,mean((teste$a1-predict(modelo4,teste[,1:11]))^2))
}

In [ ]:
metricas

In [ ]:
m4=cbind(mean(metricas),sd(metricas))
m=rbind(m,m4)

# Imputação KNN

In [ ]:
dados=AjeitaDados(dados)
apply(dados,2,function(x) sum(is.na(x)))

# 1 Vizinho mais próximo

In [ ]:
dados=knnImputation(dados,1,scale=T,meth="median")
apply(dados,2,function(x) sum(is.na(x)))

In [ ]:
metricas=c()
set.seed(7)
for(i in 1:10){
amostra=S.SI(nrow(dados),floor(nrow(dados)*0.7))
treino=dados[amostra,]
teste=dados[-amostra,]
modelo5=rpart(a1 ~ ., data = treino[, 1:12])
rpart.plot(modelo5, type=0, varlen=10)
##Avaliação do modelo
metricas=append(metricas,mean((teste$a1-predict(modelo5,teste[,1:11]))^2))
}

In [ ]:
metricas

In [ ]:
m5=cbind(mean(metricas),sd(metricas))
m=rbind(m,m5)

# K Vizinhos mais próximos

In [ ]:
#Método pela função kernel
set.seed(7)
for (i in 2:10){
  metricas=c()
  dados=AjeitaDados(dados)
  dados=knnImputation(dados,i,scale=T,meth="weighAvg")
  for(j in 1:10){
    amostra=S.SI(nrow(dados),floor(nrow(dados)*0.7))
    treino=dados[amostra,]
    teste=dados[-amostra,]
    modelo=rpart(a1 ~ ., data = treino[, 1:12])
    ##Avaliação do modelo
    metricas=append(metricas,mean((teste$a1-predict(modelo,teste[,1:11]))^2))
    m6=cbind(mean(metricas),sd(metricas))           
                }
    m=rbind(m,m6)
}

In [ ]:
#Método pela mediana
set.seed(7)
for (i in 2:10){
  metricas=c()
  dados=AjeitaDados(dados)
  dados=knnImputation(dados,i,scale=T,meth="median")
  for(j in 1:10){
    amostra=S.SI(nrow(dados),floor(nrow(dados)*0.7))
    treino=dados[amostra,]
    teste=dados[-amostra,]
    modelo=rpart(a1 ~ ., data = treino[, 1:12])
    ##Avaliação do modelo
    metricas=append(metricas,mean((teste$a1-predict(modelo,teste[,1:11]))^2))
    m6=cbind(mean(metricas),sd(metricas))           
  }
  m=rbind(m,m6)
}

# IMPUTAÇÃO MULTIPLA

In [ ]:
dados=AjeitaDados(dados)
nrow(dados)

In [ ]:
md.pattern(dados)

In [ ]:
apply(dados,2,function(x) sum(is.na(x)))

In [ ]:
imputacao=mice(dados,m=5,method="pmm",maxit=50)
summary(imputacao)

In [ ]:
##Valores imputados por banco de dados
imputacao$imp[c(4,5,6,10,11)]

In [ ]:
# Banco de dados específico
head(complete(imputacao,4))

In [ ]:
metricas2=c()
set.seed(7)
for(j in 1:10){
amostra=S.SI(nrow(dados),floor(nrow(dados)*0.7))
treino=dados[amostra,]
teste=dados[-amostra,]
metricas=c()
for (i in 1:5){
  dados=complete(imputacao,i)
  modelo=rpart(a1 ~ ., data = treino[, 1:12])
  ##Avaliação do modelo
  t=mean((teste$a1-predict(modelo,teste[,1:11]))^2)
  metricas=append(metricas,t)
              }
metricas2=append(metricas2,mean(metricas))
            }
metricas2
m7=cbind(mean(metricas2),sd(metricas2))
m=rbind(m,m7)


In [ ]:
(rownames(m)=c("rpart","Tirando linhas","Medidas de Tendência","Regressão Linear",
               "1 vizinho","2 vizinhos/Kernel","3 vizinhos/Kernel","4 vizinhos/Kernel","5 vizinhos/Kernel",
               "6 vizinhos/Kernel","7 vizinhos/Kernel","8 vizinhos/Kernel","9 vizinhos/Kernel"
               ,"10 vizinhos/Kernel","2 vizinhos/Mediana","3 vizinhos/Mediana","4 vizinhos/Mediana"
               ,"5 vizinhos/Mediana","6 vizinhos/Mediana","7 vizinhos/Mediana","8 vizinhos/Mediana"
               ,"9 vizinhos/Mediana","10 vizinhos/Mediana","I.M")) 
colnames(m)=c("Média da Performance","Desvio-Padrão da Performance")


In [ ]:
m[sort(m[,1],index.return=T)$ix,]